In [1]:
from o2s.models.object_2_sphere import Mesh2Sphere
from o2s.datasets.radar_dataset import RadarDataset

import numpy as np
import matplotlib.pyplot as plt
from torch_harmonics import plotting
from torch_geometric.loader import DataLoader
import xarray
import torch
from torch import nn

In [2]:
dataset = RadarDataset('/home/colin/hdd/workspace/datasets/test_dataset.nc', 'test', return_mesh=True)

In [3]:
B = 1
dataloader = DataLoader(dataset, batch_size=B, shuffle=True)

In [4]:
model = Mesh2Sphere(3, 25, 32, 0.5, use_mlp=False).cuda()

/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/

In [5]:
batch, target, vert, face = next(iter(dataloader))

In [6]:
batch

DataBatch(x=[63, 1], edge_index=[2, 181], pos=[63, 3], edge_vec=[181, 3], batch=[63], ptr=[2])

In [7]:
import trimesh
mesh = trimesh.Trimesh(vertices=vert[0,0], faces=face[0,0], process=True, validate=False)

In [8]:
pred, w = model(batch.cuda())

AssertionError: 

In [ ]:
plotting.plot_spherical_fn(pred.detach().cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)))

In [ ]:
plotting.plot_spherical_fn(target[0,1,:,:,0].squeeze().numpy(), fig=plt.figure(figsize=(2,2)))

In [ ]:
target = torch.view_as_real(target)[0,0,:,:,0] 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

for iter in range(1000):
    y_pred, _ = model(batch.cuda())
    loss = (y_pred - target.view(1,61,21).cuda()).pow(2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if iter % 10 == 0:
        print(f'iteration: {iter} loss: {loss.item()}')

In [ ]:
with torch.no_grad():
    pred, w = model(batch.cuda())
    plotting.plot_spherical_fn(pred.cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)))

In [ ]:
plotting.plot_spherical_fn(target.squeeze().numpy(), fig=plt.figure(figsize=(2,2)))